

During the model assessment, Test Error Rate or Accuracy cannot reflect **Variance** of the learning algorithm.
* Test Error (Prediction Error; Generalization Error)
    $$\operatorname{Err}_{\mathcal{T}}=\mathrm{E}[L(Y, \hat{f}(X)) \mid \mathcal{T}]$$ 
* Variance: indicate the variation of the test errors between different test samples (or including training sample). 
* Which setting is suitable to measure the variance?
    + Vary models; fixed training data, test data
    + Fixed model, training data; vary test data
    + Fixed training data; vary model, test data
    + Vary training data, model, test data

<!-- * Expected Test Error
    $$\operatorname{Err}=\mathrm{E}[L(Y, \hat{f}(X))]=\mathrm{E}\left[\operatorname{Err}_{\mathcal{T}}\right]$$ -->

*  the prediction error (light red curves) $\operatorname{Err}_{\mathcal{T}}$ for 100 simulated training sets each of size 50. The lasso was used to produce the sequence of fits.



<center><img src="pics/bias_variance.png" width="500"></center>

* The more variance of the fit model, the more difference between training sample and test sample.
* Actually, the gap between training sample and test sample is commonly larger than the performance gaps between test samples.

How can we use the last setting can be used to develop an ensemble model to reduce variance?